In [1]:
import torch

print("PyTorch 버전:", torch.__version__)
print("GPU 사용 가능 여부:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU 이름:", torch.cuda.get_device_name(0))


PyTorch 버전: 2.1.0+cu118
GPU 사용 가능 여부: True
GPU 이름: NVIDIA GeForce RTX 3090


In [ ]:
#!pip install torch transformers accelerate
import re
import time
import json
from transformers import AutoTokenizer, AutoModelForCausalLM


# 모델 선정

In [6]:
model_name = "microsoft/phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                            torch_dtype=torch.float16)
model.to("cuda")

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_features=32064, 

# 프롬프트

In [10]:
# 프롬프트 러닝 비교

##제로샷

def prompt_a(input_text):
    return f"""
다음 문장에서 사용자의 의도(intent), 필요한 정보(slot), 그리고 자연스러운 응답 메시지를 추출하세요.  
출력은 아래 형식을 따르세요:

{
  "intent": "...",
  "slots": { "recipient": "...", "amount": ... },
  "response": "..."
}

문장: "{input_text}"
"""


def prompt_b(input_text):
    return f"""
    다음 문장에서 사용자의 의도(intent), 필요한 정보(slot), 그리고 응답 메시지를 추출하세요.
    의도(intent)는 아래 분류 중 하나로만 선택해야 합니다:
    다음 문장을 분석하여 intent, amount, recipient, response를 JSON형식으로 출력해 주세요.

    - transfer: 금전을 누군가에게 보내려는 요청
    - inquiry: 잔액, 수수료 등 정보 문의
    - cancel: 진행 중인 작업 취소
    - confirm: 이전 발화에 대한 확인
    - other: 시스템 외 일상 대화 또는 분류 불가능
    - system_response: 시스템이 사용자에게 안내하거나 재질문

    출력은 아래 JSON 형식을 따르세요:

{
  "intent": "...",
  "slots": { "recipient": "...", "amount": ... },
  "response": "..."
}

문장: "{input_text}"
"""

# 함수 정의

In [21]:
# run_inference.py
def run_inference(prompt: str, tokenizer, model):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    start = time.time()
    outputs = model.generate(**inputs, max_new_tokens=80,do_sample=False)
    end = time.time()

    # 출력 해석
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output_text = generated.replace(prompt, "").strip()


    try:
        json_start = output_text.find("{")
        json_end = output_text.rfind("}") + 1
        json_block = output_text[json_start:json_end]
        parsed = json.loads(json_block)
        
    except Exception as e:
        print("JSON 파싱 실패:", e)
        print("📦 원문 출력:", repr(output_text))
        parsed = {}

    print("\n📦 LLM 원문 출력:")
    print(output_text)
    print(f"\n⏱️ 처리 시간: {round(end - start, 2)}초")
    return parsed, round(end - start, 2)


In [50]:
def evaluate_results(results_path, ground_truth):
    import json

    with open(results_path, "r", encoding="utf-8") as f:
        results = json.load(f)

    def clean(x): return str(x).strip().lower() if x else ""
    def to_int(x):
        try: return int(str(x).replace(",", "").strip())
        except: return None

    total = len(results)

    intent_acc = sum(clean(r["intent"]) == clean(gt["intent"]) for r, gt in zip(results, ground_truth)) / total
    amount_acc = sum(to_int(r["slots"].get("amount")) == gt["slots"].get("amount") for r, gt in zip(results, ground_truth)) / total
    recipient_acc = sum(clean(r["slots"].get("recipient")) == clean(gt["slots"].get("recipient")) for r, gt in zip(results, ground_truth)) / total
    avg_time = sum(r.get("inference_time", 0) for r in results) / total
    json_success = sum(bool(r.get("intent")) for r in results) / total

    return {
        "intent_acc": round(intent_acc, 4),
        "amount_acc": round(amount_acc, 4),
        "recipient_acc": round(recipient_acc, 4),
        "json_success_rate": round(json_success, 4),
        "avg_response_time": round(avg_time, 2)
    }


In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# from prompt_templates import prompt_a
# from run_inference import run_inference

# # 1. 모델 로딩
# model_name = "microsoft/phi-3-mini-4k-instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# model.to("cuda")

# 2. 입력 문장
text = "아빠한테 오천 원 보내줘"
prompt = prompt_a(text)

# 3. 단독 인퍼런스 실행
result = run_inference(prompt, tokenizer, model)

print("\n🎯 최종 출력 결과:")
print(result)


📦 LLM 원문 출력:
### Solution:

{"intent": "transfer","amount": 100000,"recipient": "엄마","response": "엄마님께 100,000원을 송금해드릴까요?"}


##

⏱️ 처리 시간: 2.49초

🎯 최종 출력 결과:
{'intent': 'transfer', 'amount': 100000, 'recipient': '엄마', 'response': '엄마님께 100,000원을 송금해드릴까요?'}


# Model Test

In [18]:
samples = [
  { "text": "철수한테 오천 원 보내줘", "intent": "transfer", "slots": { "recipient": "철수", "amount": 5000 } },
  { "text": "엄마한테 3만원만", "intent": "transfer", "slots": { "recipient": "엄마", "amount": 30000 } },
  { "text": "만원만 보내줘", "intent": "transfer", "slots": { "recipient": None, "amount": 10000 } },
  { "text": "엄마한테 보내줘", "intent": "transfer", "slots": { "recipient": "엄마", "amount": None } },
  { "text": "나 송금하고 싶어", "intent": "transfer", "slots": { "recipient": None, "amount": None } },
  { "text": "가스비 내야 하는데 어떻게 해야 해?", "intent": "inquiry", "slots": { "topic": "가스비" } },
  { "text": "내 계좌에 얼마 남았어?", "intent": "inquiry", "slots": { "topic": "잔액조회" } },
  { "text": "그 사람한테 또 보내줘", "intent": "transfer", "slots": { "recipient": "이전대화", "amount": "이전대화" } },
  { "text": "아까랑 똑같이 보내줘", "intent": "transfer", "slots": { "recipient": "이전대화", "amount": "이전대화" } },
  { "text": "송금할래", "intent": "transfer", "slots": { "recipient": None, "amount": None } },
  { "text": "이체 수수료 얼마야?", "intent": "inquiry", "slots": { "topic": "이체 수수료" } },
  { "text": "오늘 아빠랑 점심 먹었어", "intent": "other", "slots": {} },
  { "text": "아빠한테 이만원 보내줘", "intent": "transfer", "slots": { "recipient": "아빠", "amount": 20000 } },
  { "text": "여보한테 십만 원 이체해줘", "intent": "transfer", "slots": { "recipient": "여보", "amount": 100000 } },
  { "text": "보내지 마", "intent": "cancel", "slots": {} },
  { "text": "그냥 넘어가줘", "intent": "cancel", "slots": {} },
  { "text": "엄마 송금해줘", "intent": "transfer", "slots": { "recipient": "엄마", "amount": None } },
  { "text": "아, 삼만원 보내는 거였지", "intent": "confirm", "slots": { "amount": 30000 } },
  { "text": "예금해줘", "intent": "other", "slots": { "action": "예금" } },
  { "text": "누구한테 보낼까요?", "intent": "system_response", "slots": {} }
]


In [26]:
# from run_inference import run_inference
import json

results = []

for sample in samples:  # samples는 [{text: ..., intent: ...}, ...] 형식
    prompt = prompt_b(sample["text"])  # 너가 만든 프롬프트 함수
    output, elapsed = run_inference(prompt, tokenizer, model)

    results.append({
        "input_text": sample["text"],
        "expected_intent": sample["intent"],
        "predicted_intent": output.get("intent", "None"),
        "slots": output.get("slots", {}),
        "response": output.get("response", ""),
        "inference_time": elapsed
    })

# 결과 저장
with open("results1.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("✅ 결과 저장 완료: results.json")



📦 LLM 원문 출력:
{"intent": "transfer","amount": 100000,"recipient": "철수","response": "철수님께 천원을 송금해드릴까요?"}  

text: "저는 친구가

⏱️ 처리 시간: 2.61초

📦 LLM 원문 출력:
{"intent": "transfer","amount": 30000,"recipient": "엄마","response": "엄마님께 3만원을 송금해드릴까요?"}  

text: "저는 엄마에�

⏱️ 처리 시간: 2.54초

📦 LLM 원문 출력:
{"intent": "transfer","amount": 10000,"recipient": "엄마","response": "엄마님께 10,000원을 송금해드릴까요?"}  

text: "저는 ��

⏱️ 처리 시간: 2.51초
JSON 파싱 실패: Expecting value: line 1 column 33 (char 32)
📦 원문 출력: '{"intent": "transfer","amount": None,"recipient": "엄마","response": "엄마님께 얼마를 송금하시겠습니까?"}  \n\ntext: "저는 엄마에��'

📦 LLM 원문 출력:
{"intent": "transfer","amount": None,"recipient": "엄마","response": "엄마님께 얼마를 송금하시겠습니까?"}  

text: "저는 엄마에��

⏱️ 처리 시간: 2.5초
JSON 파싱 실패: Expecting value: line 1 column 32 (char 31)
📦 원문 출력: '{"intent": "inquiry","amount": None,"recipient": None,"response": "누구나 송금하려면 얼마를 예금하시겠습니까?"}  \n\ntext: "저는 송'

📦 LLM 원문 출력:
{"intent": "inquiry","amount": None,"recipient": None,"response": "누구나 송금하려면

In [55]:
result_a = evaluate_results("results.json", samples)
result_b = evaluate_results("results1.json", samples)



KeyError: 'intent'

In [47]:
def evaluate_results(results_path):
    import json

    with open(results_path, "r", encoding="utf-8") as f:
        results = json.load(f)

    def clean(x): return str(x).strip().lower() if x else ""
    def to_int(x):
        try: return int(str(x).replace(",", "").strip())
        except: return None

    total = len(results)

    intent_correct = 0
    amount_correct = 0
    recipient_correct = 0
    json_parsed = 0
    total_time = 0
    slot_total = 0

    for r in results:
        if not r.get("predicted_intent"):  # JSON 파싱 실패
            continue

        json_parsed += 1
        total_time += r.get("inference_time", 0)

        # Intent 정확도
        if clean(r.get("predicted_intent")) == clean(r.get("expected_intent")):
            intent_correct += 1

        # Slot 정확도 평가 (optional: if expected_slots 포함)
        if "expected_slots" in r:
            expected = r["expected_slots"]
            predicted = r.get("slots", {})

            if "amount" in expected:
                if to_int(predicted.get("amount")) == to_int(expected.get("amount")):
                    amount_correct += 1
                slot_total += 1

            if "recipient" in expected:
                if clean(predicted.get("recipient")) == clean(expected.get("recipient")):
                    recipient_correct += 1
                slot_total += 1

    return {
        "intent_acc": round(intent_correct / total, 4),
        "amount_acc": round(amount_correct / total, 4) if total else "N/A",
        "recipient_acc": round(recipient_correct / total, 4) if total else "N/A",
        "json_success_rate": round(json_parsed / total, 4),
        "avg_response_time": round(total_time / json_parsed, 2) if json_parsed else 0
    }



In [28]:
for i,ex in enumerate(sample):
    input_text = ex["text"]
    print(f"\n[{i+1}] 문장: {input_text}")

    result = run_chatbot_inference(input_text) 


[1] 문장: 철수한테 오천 원 보내줘
✅ 파싱된 결과:
{
  "intent": "transfer",
  "amount": 100000,
  "recipient": "철수",
  "response": "철수님께 천원을 송금해드릴까요?"
}

⏱️ 처리 시간: 4.07초

[2] 문장: 엄마한테 3만원만
❌ JSON 파싱 실패: Expecting value: line 1 column 1 (char 0)
📦 원문 출력: 'text: "저는 송금 예정이에요"\n\n  {"intent": "inquiry","amount": None,"recipient": None,"response": "송금 예정이에요네요, 언제 송금'

⏱️ 처리 시간: 3.91초

[3] 문장: 만원만 보내줘
❌ JSON 파싱 실패: Expecting value: line 1 column 52 (char 51)
📦 원문 출력: '{"intent": "transfer","amount": 10000,"recipient": None,"response": "10,000원을 송금하시겠습니까?"}\n\n\n송금하고 싶어요\n\n  {"intent": "'

⏱️ 처리 시간: 3.94초

[4] 문장: 엄마한테 보내줘
✅ 파싱된 결과:
{
  "intent": "transfer",
  "amount": 10000,
  "recipient": "엄마",
  "response": "엄마님께 10,000원을 송금해드릴까요?"
}

⏱️ 처리 시간: 4.09초

[5] 문장: 나 송금하고 싶어
❌ JSON 파싱 실패: Expecting value: line 1 column 1 (char 0)
📦 원문 출력: ''

⏱️ 처리 시간: 4.11초

[6] 문장: 가스비 내야 하는데 어떻게 해야 해?
❌ JSON 파싱 실패: Expecting value: line 1 column 32 (char 31)
📦 원문 출력: '{"intent": "inquiry","amount": None,"recipient": None,"

In [10]:
correct_intent = 0
correct_recipient = 0
correct_amount = 0
    

    pred_intent = result.get("intent")
    pred_recipient = result.get("recipient")
    pred_amount = int(result.get("amount")) if result.get("amount") is not None else None

    # 의도 비교
    if pred_intent == ex["intent"]:
        correct_intent += 1
        print("✔ intent 정답")
    else:
        print(f"❌ intent 오답 (예상: {ex['intent']}, 예측: {pred_intent})")

    # 대상 비교
    if pred_recipient == ex["slots"]["recipient"]:
        correct_recipient += 1
        print("✔ recipient 정답")
    else:
        print(f"❌ recipient 오답 (예상: {ex['slots']['recipient']}, 예측: {pred_recipient})")

    # 금액 비교
    if pred_amount == ex["slots"]["amount"]:
        correct_amount += 1
        print("✔ amount 정답")
    else:
        print(f"❌ amount 오답 (예상: {ex['slots']['amount']}, 예측: {pred_amount})")

# 평가 결과 출력
total = len(sample)
print("\n📊 평가 결과")
print(f"Intent 정확도: {correct_intent}/{total} ({correct_intent/total:.0%})")
print(f"Recipient 정확도: {correct_recipient}/{total} ({correct_recipient/total:.0%})")
print(f"Amount 정확도: {correct_amount}/{total} ({correct_amount/total:.0%})")

❌ intent 오답 (예상: 시스템응답, 예측: None)


KeyError: 'recipient'

In [ ]:
import torch
torch.cuda.empty_cache()


In [57]:
# 재실행 후 다시 samples와 저장 로직 복원
samples = [
  {
    "text": "철수한테 오천 원 보내줘",
    "intent": "transfer",
    "slots": { "recipient": "철수", "amount": 5000 },
    "response": "철수님께 5,000원을 송금하시겠어요?"
  },
  {
    "text": "엄마한테 3만원만",
    "intent": "transfer",
    "slots": { "recipient": "엄마", "amount": 30000 },
    "response": "엄마님께 30,000원을 송금하시겠어요?"
  },
  {
    "text": "만원만 보내줘",
    "intent": "transfer",
    "    slots": { "recipient": None, "amount": 10000 },
    "response": "누구에게 10,000원을 송금할까요?"
  },
  {
    "text": "엄마한테 보내줘",
    "intent": "transfer",
    "slots": { "recipient": "엄마", "amount": None },
    "response": "엄마님께 얼마를 송금할까요?"
  },
  {
    "text": "나 송금하고 싶어",
    "intent": "transfer",
    "slots": { "recipient": None, "amount": None },
    "response": "송금하실 대상과 금액을 알려주세요."
  },
  {
    "text": "가스비 내야 하는데 어떻게 해야 해?",
    "intent": "inquiry",
    "slots": { "topic": "가스비" },
    "response": "가스비 납부 방법을 알려드릴게요."
  },
  {
    "text": "내 계좌에 얼마 남았어?",
    "intent": "inquiry",
    "slots": { "topic": "잔액조회" },
    "response": "계좌 잔액을 확인 중입니다."
  },
  {
    "text": "그 사람한테 또 보내줘",
    "intent": "transfer",
    "slots": { "recipient": "이전대화", "amount": "이전대화" },
    "response": "이전에 송금한 대상과 금액으로 다시 송금할까요?"
  },
  {
    "text": "아까랑 똑같이 보내줘",
    "intent": "transfer",
    "slots": { "recipient": "이전대화", "amount": "이전대화" },
    "response": "이전과 동일한 내용으로 송금 진행할까요?"
  },
  {
    "text": "송금할래",
    "intent": "transfer",
    "slots": { "recipient": None, "amount": None },
    "response": "송금하실 대상과 금액을 말씀해주세요."
  },
  {
    "text": "이체 수수료 얼마야?",
    "intent": "inquiry",
    "slots": { "topic": "이체 수수료" },
    "response": "이체 수수료 정보를 알려드릴게요."
  },
  {
    "text": "오늘 아빠랑 점심 먹었어",
    "intent": "other",
    "slots": {},
    "response": "말씀 감사합니다!"
  },
  {
    "text": "아빠한테 이만원 보내줘",
    "intent": "transfer",
    "slots": { "recipient": "아빠", "amount": 20000 },
    "response": "아빠님께 20,000원을 송금하시겠어요?"
  },
  {
    "text": "여보한테 십만 원 이체해줘",
    "intent": "transfer",
    "slots": { "recipient": "여보", "amount": 100000 },
    "response": "여보님께 100,000원을 송금하시겠어요?"
  },
  {
    "text": "보내지 마",
    "intent": "cancel",
    "slots": {},
    "response": "요청하신 송금을 취소했습니다."
  },
  {
    "text": "그냥 넘어가줘",
    "intent": "cancel",
    "slots": {},
    "response": "진행 중이던 요청을 취소했어요."
  },
  {
    "text": "엄마 송금해줘",
    "intent": "transfer",
    "slots": { "recipient": "엄마", "amount": None },
    "response": "엄마님께 얼마를 송금할까요?"
  },
  {
    "text": "아, 삼만원 보내는 거였지",
    "intent": "confirm",
    "slots": { "amount": 30000 },
    "response": "30,000원 송금 요청으로 확인했습니다."
  },
  {
    "text": "예금해줘",
    "intent": "other",
    "slots": { "action": "예금" },
    "response": "예금을 진행할까요? 금액을 알려주세요."
  },
  {
    "text": "누구한테 보낼까요?",
    "intent": "system_response",
    "slots": {},
    "response": "누구에게 송금할까요?"
  }
]

import json
from pathlib import Path

save_path = Path("samples_with_response.json")
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(samples, f, indent=2, ensure_ascii=False)

save_path


PosixPath('samples_with_response.json')